In [70]:
#Apply for natural language model
import torch
import torch.nn as nn

class LSTM(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(LSTM,self).__init__()
    self.input_size=input_size #feature length of inputs ( Note that : inputs != input)
    self.hidden_size=hidden_size
    self.output_size=output_size
    
    #ft
    self.neuralNet_f=nn.Linear(154,128)

    #it
    self.neuralNet_i=nn.Linear(154,128)

    #ct
    self.neuralNet_c=nn.Linear(154,128)

    #ot
    self.neuralNet_o=nn.Linear(154,128)

    #logits
    self.logits=nn.Linear(128,2)
  
  def forward(self,inputs):
    batch_size,sequence_length,input_size=inputs.size()
    h_t=torch.zeros((batch_size,self.hidden_size))
    c_t=torch.zeros((batch_size,self.hidden_size))

    for input_index_t in range(sequence_length):
      x_t=inputs[:,input_index_t,:]
      cat_t=torch.cat((x_t,h_t),1)

      #ft
      f_t=torch.sigmoid(self.neuralNet_f(cat_t))
      
      #it
      i_t=torch.sigmoid(self.neuralNet_i(cat_t))

      #c~t (sign as gt)
      g_t=torch.tanh(self.neuralNet_c(cat_t))

      #ot
      o_t=torch.sigmoid(self.neuralNet_o(cat_t))

      #C_t
      c_t=f_t*c_t+i_t*g_t
      
      #h_t
      h_t=o_t*torch.tanh(c_t)
      output=self.logits(h_t)

    return h_t,c_t,output



In [72]:
batch_size=64
sequence_length=5
feature_length=26 #input_size
hidden_size=128
inputs=torch.ones((batch_size,sequence_length,feature_length)) 

hidden=torch.zeros((batch_size,hidden_size))

cat=torch.cat((inputs[:,0,:],hidden),1)
print(cat.shape)

lstm=LSTM(26,128,2)
output=lstm(inputs)[2]
print(output.shape)

torch.Size([64, 154])
torch.Size([64, 2])
